# Couche gold
Ajout des colonnes `featuring` (True/False) et `nb_artistes` (nombre d'artistes) à partir des données Silver.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType, IntegerType
import ast
import os

Démarrer la session Spark

In [7]:
#spark = SparkSession.builder.appName("SilverToGold").getOrCreate()

spark = SparkSession.builder \
    .appName("SilverToGold") \
    .getOrCreate()

Définir les chemins d'accès

In [8]:
SILVER_PATH = "../Silver/data/musicInfo"
GOLD_PATH = "../gold_data.csv"

Définition des UDF pour compter les artistes et détecter un featuring

In [9]:
def count_artists(artists_str):
    try:
        artists = ast.literal_eval(artists_str)
        return len(artists)
    except:
        return 1

def is_featuring(artists_str):
    try:
        artists = ast.literal_eval(artists_str)
        return len(artists) > 1
    except:
        return False

count_artists_udf = udf(count_artists, IntegerType())
is_featuring_udf = udf(is_featuring, BooleanType())

Lecture des données Silver

In [10]:
# Vérifier si le répertoire existe et contient des fichiers
print(os.path.exists(SILVER_PATH))
print("Contenu du répertoire Silver :")
print(os.listdir(SILVER_PATH))

df = spark.read.parquet(SILVER_PATH)
df.show(5)
df.printSchema()

True
Contenu du répertoire Silver :
['.part-00000-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00001-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00002-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00003-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00004-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00005-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00006-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '.part-00007-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet.crc', '._SUCCESS.crc', 'part-00000-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet', 'part-00001-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet', 'part-00002-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet', 'part-00003-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.snappy.parquet', 'part-00004-59253d0c-cce7-4265-9ade-0bd2e1f9238f-c000.

Ajout des colonnes `nb_artistes` et `featuring`

In [11]:
df_gold = df.withColumn("nb_artistes", count_artists_udf(col("artists"))) \
            .withColumn("featuring", is_featuring_udf(col("artists")))

df_gold.show()

Py4JJavaError: An error occurred while calling o112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$ma

Affichage d'un aperçu

In [ ]:
df_gold.select("artists", "nb_artistes", "featuring").show(10, truncate=False)

Py4JJavaError: An error occurred while calling o70.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$map

Sauvegarde au format CSV

In [ ]:
df_gold.write.mode("overwrite").option("header", "true").csv(GOLD_PATH)
print(f"Données gold sauvegardées dans {GOLD_PATH} avec les colonnes nb_artistes et featuring.")

Py4JJavaError: An error occurred while calling o152.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 7.0 failed 1 times, most recent failure: Lost task 2.0 in stage 7.0 (TID 9) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartit

Arrêt de la session Spark

In [ ]:
spark.stop()